In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
import cv2

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
import warnings
from PIL import Image, ImageFilter
warnings.simplefilter('ignore')

训练数据读取

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train

image_id - 图像的 ID 代码，格式为 '{video_id}-{video_frame}' 注释 - 任何海星检测的边界框，字符串格式，可以直接用 Python 评估。

In [ ]:
train.annotations.unique()

In [ ]:
test = pd.read_csv('../input/tensorflow-great-barrier-reef/test.csv')
test

提交文件

In [ ]:
sub = pd.read_csv('../input/tensorflow-great-barrier-reef/example_sample_submission.csv')
sub

In [ ]:
sample_test = np.load('../input/tensorflow-great-barrier-reef/example_test.npy') 
sample_test[0][0]

In [ ]:
fig = plt.figure(figsize=(30,20))
for i in range(3):
    fig.add_subplot(4, 5, i+1)
    plt.title(i)
    plt.imshow(sample_test[i])

预计在测试集中看到大约 13,000 张图像。

In [ ]:
len(train)

In [ ]:
anno = train[train.annotations!='[]']
anno

按视频划分数据

In [ ]:
video0 = anno[anno.video_id==0].reset_index(drop=True)
video0

In [ ]:
video1 = anno[anno.video_id==1].reset_index(drop=True)
video1

In [ ]:
video2 = anno[anno.video_id==2].reset_index(drop=True)
video2

In [ ]:
a = video0.annotations.loc[0][2:-2]
a = a.split(',')
a

In [ ]:
x = int(a[0].split(':')[1])
y = int(a[1].split(':')[1])
width = int(a[2].split(':')[1])
height = int(a[3].split(':')[1])

In [ ]:
fig = plt.figure(figsize=(30,20))

video_imid = ('../input/tensorflow-great-barrier-reef/train_images/video_0/16' +'.jpg')
img = cv2.imread(video_imid)

img = cv2.rectangle(img,
              pt1=(x, y),
              pt2=(x+height, y-width),
              color=(0, 255, 0),
              thickness=3,
              lineType=cv2.LINE_4,
              shift=0)

plt.imshow(img)

把注解和照片合并起来展示一下

In [ ]:
for i in range(15):
    video0[f'x_{i}'] =str()
    video0[f'y_{i}'] = str()
    video0[f'width_{i}'] = str()
    video0[f'height_{i}'] = str()
video0['path'] =str()

for x in video0.index:
    a = video0.annotations.loc[x][2:-2]
    a = a.split(',')
    
    for i in range(int(len(a)/4)):
        video0[f'x_{i}'][x] = int(a[0].split(':')[1])
        video0[f'y_{i}'][x] = int(a[1].split(':')[1])
        video0[f'width_{i}'][x] = int(a[2].split(':')[1])
        video0[f'height_{i}'][x] = int(a[3].split(':')[1].split('}')[0])
    video0['path'][x] = '../input/tensorflow-great-barrier-reef/train_images/video_0/' + str(video0.video_id.loc[i]) +'.jpg'

In [ ]:
fig = plt.figure(figsize=(30,20))
for i in range(12):
    fig.add_subplot(4, 3, i+1)
    plt.title(video0.image_id[i])
    img = cv2.imread(video0.path[i])
    for z in range(15):
        if video0[f'x_{z}'][i] !=str():
            x = video0[f'x_{z}'][i]
            y = video0[f'x_{z}'][i]
            width=  video0[f'width_{z}'][i]
            height= video0[f'height_{z}'][i]
            img = cv2.rectangle(img,
              pt1=(x, y),
              pt2=(x+height, y-width),
              color=(0, 255, 0),
              thickness=3,
              lineType=cv2.LINE_4,
              shift=0)

    plt.imshow(img)


In [ ]:
for i in range(25):
    video1[f'x_{i}'] =str()
    video1[f'y_{i}'] = str()
    video1[f'width_{i}'] = str()
    video1[f'height_{i}'] = str()
video1['path'] =str()

for x in video1.index:
    a = video1.annotations.loc[x][2:-2]
    a = a.split(',')
    
    for i in range(int(len(a)/4)):
        video1[f'x_{i}'][x] = int(a[0].split(':')[1])
        video1[f'y_{i}'][x] = int(a[1].split(':')[1])
        video1[f'width_{i}'][x] = int(a[2].split(':')[1])
        video1[f'height_{i}'][x] = int(a[3].split(':')[1].split('}')[0])
    video1['path'][x] = '../input/tensorflow-great-barrier-reef/train_images/video_0/' + str(video0.video_id.loc[i]) +'.jpg'

In [ ]:
video1[video1[f'x_{17}'] !=str()] 

In [ ]:
i=1759
for z in range(25):
            x = video1[f'x_{z}'][i]
            y = video1[f'x_{z}'][i]
            width=  video1[f'width_{z}'][i]
            height= video1[f'height_{z}'][i]
            print(x,y,width,height)

In [ ]:
i=1760
for z in range(25):
            x = video1[f'x_{z}'][i]
            y = video1[f'x_{z}'][i]
            width=  video1[f'width_{z}'][i]
            height= video1[f'height_{z}'][i]
            print(x,y,width,height)

In [ ]:
fig = plt.figure(figsize=(30,20))
for i in range(12):
    fig.add_subplot(4, 3, i+1)
    plt.title(video1.image_id[i])
    img = cv2.imread(video1.path[i])
    for z in range(25):
        if video1[f'x_{z}'][i] !=str():
            x = video1[f'x_{z}'][i]
            y = video1[f'x_{z}'][i]
            width=  video1[f'width_{z}'][i]
            height= video1[f'height_{z}'][i]
            img = cv2.rectangle(img,
              pt1=(x, y),
              pt2=(x+height, y-width),
              color=(0, 255, 0),
              thickness=3,
              lineType=cv2.LINE_4,
              shift=0)
    print(x,y,width,height)

    plt.imshow(img)

In [ ]:
for i in range(25):
    video2[f'x_{i}'] =str()
    video2[f'y_{i}'] = str()
    video2[f'width_{i}'] = str()
    video2[f'height_{i}'] = str()
video2['path'] =str()

for x in video2.index:
    a = video2.annotations.loc[x][2:-2]
    a = a.split(',')
    
    for i in range(int(len(a)/4)):
        video2[f'x_{i}'][x] = int(a[0].split(':')[1])
        video2[f'y_{i}'][x] = int(a[1].split(':')[1])
        video2[f'width_{i}'][x] = int(a[2].split(':')[1])
        video2[f'height_{i}'][x] = int(a[3].split(':')[1].split('}')[0])
    video2['path'][x] = '../input/tensorflow-great-barrier-reef/train_images/video_0/' + str(video0.video_id.loc[i]) +'.jpg'

In [ ]:
fig = plt.figure(figsize=(30,20))
for i in range(12):
    fig.add_subplot(4, 3, i+1)
    plt.title(video2.image_id[i])
    img = cv2.imread(video2.path[i])
    for z in range(25):
        if video2[f'x_{z}'][i] !=str():
            x = video2[f'x_{z}'][i]
            y = video2[f'x_{z}'][i]
            width=  video2[f'width_{z}'][i]
            height= video2[f'height_{z}'][i]
            img = cv2.rectangle(img,
              pt1=(x, y),
              pt2=(x+height, y-width),
              color=(0, 255, 0),
              thickness=3,
              lineType=cv2.LINE_4,
              shift=0)

    plt.imshow(img)